Throughout the rest of the book, I use the following import conventions for NumPy and pandas:

In [1]:
import numpy as np
import pandas as pd

You may also find it easier to import Series and DataFrame into the local namespace since they are so frequently used:

In [2]:
from pandas import Series, DataFrame

# 1 Introduction to pandas

## 1.1 Series

A Series is a one-dimentional array-like object containing a sequence of values of the same type and an associated array of data labels, called its index.  

`pandas.Series` 就是带标签（index）的一维数组。可以理解成：
> Excel 里的一列（含行号和数据）

In [3]:
obj = pd.Series([4, 7, -5, 3])

obj

0    4
1    7
2   -5
3    3
dtype: int64

You can get the array representation and index object of the Series via its `array` and `index` attributes, respectively:

In [4]:
obj.array

<NumpyExtensionArray>
[4, 7, -5, 3]
Length: 4, dtype: int64

In [5]:
obj.index

RangeIndex(start=0, stop=4, step=1)

Often, you'll want to create a Series with an index identifying each data point with a label:

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])

obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [7]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

Compared with NumPy arrays, you can use labels in the index when selecting single values or a set of values:

In [8]:
obj2["a"]

np.int64(-5)

In [11]:
obj2["d"] = 6

obj2[["c", "a", "d"]]

c    3
a   -5
d    6
dtype: int64

Using NumPy functions or NumPy-like operations, such as filtering with a Boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [12]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [13]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [14]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Another way to think about a Series is as a fixed-length, ordered dictionary, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dictionary:

In [15]:
"b" in obj2

True

In [16]:
"e" in obj2

False

Should you have data contained in a Python dictionary, you can create a Series from it by passing the dictionary:

In [17]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}

obj3 = pd.Series(sdata)

obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

A Series can be converted back to a dictionary with its to_dict method:

In [18]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

You can override this by passing an index with the dictionary keys in the order you want them to appear in the resulting Series:

In [19]:
states = ["California", "Ohio", "Oregon", "Texas"]

obj4 = pd.Series(sdata, index=states)

obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

> Here, three values found in sdata were placed in the appropriate locations, but since no value for "California" was found, it appears as NaN (Not a Number), which is considered in pandas to mark missing or NA values. Since "Utah" was not included in states, it is excluded from the resulting object.

I will use the terms “missing,” “NA,” or “null” interchangeably to refer to missing data. The `isna` and `notna` functions in pandas should be used to detect missing data:

In [22]:
pd.isna(obj4) # Alter: obj4.isna()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [23]:
pd.notna(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [24]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Both the Series object itself and its index have a `name` attribute, which integrates with other areas of pandas functionality:

In [25]:
obj4.name = "population"

obj4.index.name = "state"

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

A Series's index can be altered in place by assignment:

In [26]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [27]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]

obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## 1.2 DataFrame

`pd.DataFrame` 是由多个 `Series` 按列组合而成的二维表格结构。
可以理解成：
> Excel 里的整个表格（含多列、多行）

There are many ways to construct a DataFrame, though one of the most common is from a dictionary of equal-length lists or NumPy arrays:

In [28]:
data = {
    "state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    "year": [2000, 2001, 2002, 2001, 2002, 2003],
    "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}

frame = pd.DataFrame(data)

frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


For large DataFrames, the `head` method selects only the **first five rows**:

In [29]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


Similarly, `tail` returns the last five rows:

In [30]:
frame.tail()

,state,year,pop
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order:

In [31]:
pd.DataFrame(data, columns=["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


If you pass a column that isn’t contained in the dictionary, it will appear with missing values in the result:

In [32]:
frame2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


A column in a DataFrame can be retrieved as a Series either by dictionary-like notation or by using the dot attribute notation:

In [33]:
frame2["state"]

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [34]:
frame2.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

Rows can also be retrieved by position or name with the special `iloc` and `loc` attributes

In [35]:
frame2.loc[1]

year     2001
state    Ohio
pop       1.7
debt      NaN
Name: 1, dtype: object

In [36]:
frame2.iloc[2]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: 2, dtype: object

Columns can be modified by assignment. For example, the empty `debt` column could be assigned a scalar value or an array of values:

In [37]:
frame2["debt"] = 16.5

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,16.5
1,2001,Ohio,1.7,16.5
2,2002,Ohio,3.6,16.5
3,2001,Nevada,2.4,16.5
4,2002,Nevada,2.9,16.5
5,2003,Nevada,3.2,16.5


In [38]:
frame2["debt"] = np.arange(6.0)

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,0.0
1,2001,Ohio,1.7,1.0
2,2002,Ohio,3.6,2.0
3,2001,Nevada,2.4,3.0
4,2002,Nevada,2.9,4.0
5,2003,Nevada,3.2,5.0


When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any index values not present:

In [39]:
val = pd.Series([-1.2, -1.5, -1.7], index=[2, 4, 5])

frame2["debt"] = val

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,-1.2
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,-1.5
5,2003,Nevada,3.2,-1.7


> Assigning a column that doesn’t exist will create a new column.

The `del` keyword will delete columns like with a dictionary. As an example, I first add a new column of Boolean values where the `state` column equals `"Ohio"`:

In [40]:
frame2["eastern"] = frame2["state"] == "Ohio"

frame2

,year,state,pop,debt,eastern
0,2000,Ohio,1.5,NaN,True
1,2001,Ohio,1.7,NaN,True
2,2002,Ohio,3.6,-1.2,True
3,2001,Nevada,2.4,NaN,False
4,2002,Nevada,2.9,-1.5,False
5,2003,Nevada,3.2,-1.7,False


The `del` method can then be used to remove this column:

In [41]:
del frame2["eastern"]

frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Another common form of data is a nested dictionary of dictionaries:

In [42]:
populations = {
    "Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
    "Nevada": {2001: 2.4, 2002: 2.9}
}

frame3 = pd.DataFrame(populations)

frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [43]:
frame3.T

,2000,2001,2002
Ohio,1.5,1.7,3.6
Nevada,NaN,2.4,2.9


The keys in the inner dictionaries are combined to form the index in the result. This isn’t true if an explicit index is specified:

In [44]:
pd.DataFrame(populations, index=[2001, 2002, 2003])

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9
2003,NaN,NaN


Dictionaries of Series are treated in much the same way:

In [45]:
pdata = {
    "Ohio": frame3["Ohio"][:-1],
    "Nevada": frame3["Nevada"][:2]
}

pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [46]:
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [47]:
frame3.index.name = "year"
frame3.columns.name = "state"

frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


Unlike Series, DataFrame does not have a `name` attribute. DataFrame's `to_numpy` method returns the data contained in the DataFrame as a two-dimensional ndarray:

In [48]:
frame3.to_numpy()

array([[1.5, nan],
       [1.7, 2.4],
       [3.6, 2.9]])

If the DataFrame’s columns are different data types, the data type of the returned array will be chosen to accommodate all of the columns:

In [49]:
frame2.to_numpy()

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, -1.2],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, -1.5],
       [2003, 'Nevada', 3.2, -1.7]], dtype=object)

## 1.3 Index Objects